In [3]:
import serial
import serial.tools.list_ports
from datetime import datetime

In [4]:
ports = serial.tools.list_ports.comports(include_links=False)

if ports:
    PORT = ports[0].device

print(PORT)

COM15


In [10]:
class FPGA_communication:
        def __init__(self, port, baud_rate):
            self.port = port
            self.baud_rate = baud_rate
        
        def open_instrument(self):
            self.ser = serial.Serial(self.port, self.baud_rate, timeout=1)
            self.log = open("log_fpga_communication.txt","a+")
            current_time = datetime.now()
            self.log.write(f"[INFO] - - - {current_time} - - - \n")
            self.log.write(f"[INFO] FPGA initialized on port {self.port}, baud rate: {self.baud_rate}\n")

        def set_memory_addr(self):
            send = self.ser.write(b"A00")
            self.log.write(f"[DEBUG] Sending memory adress\n")

            ret = self.ser.read(4)
            if (ret == b"\x00OK\n"  or ret == b"\nOK\n"):
                self.log.write(f"[INFO] Memory value successfully written at address 0x0000\n")
                return True
            else:
                self.log.write(f"[WARNING] Bad result during set memory adress\n")
                return False
            
        def write_val_mem(self, message:str):
            send = self.ser.write(b"W" + bytes(message, "utf-8"))
            if(send != 3):
                 self.log.write(f"[WARNING] Invalid message\n")
            else:
                self.log.write(f"[DEBUG] Message is valid  \n")


            ret = self.ser.read(4)
            if (ret == b"\nOK\n"):
                self.log.write(f"[INFO] Value successfully written at address 0x{message}\n")
                return True
            else:
                self.log.write(f"[WARNING] Bad result during writing value\n")
                return False
             
        def display_mem_vals_leds(self):
            send = self.ser.write(b"G")
            self.log.write(f"[DEBUG] Beginning go sequence\n")

            ret = self.ser.read(4)
            if (ret == b"\nOK\n"):
                self.log.write(f"[INFO] Go sequence successfully completed\n")
                return True
            else:
                self.log.write(f"[WARNING] Bad result during go sequence\n")
                return False
            
        def read_mem(self):
            send = self.ser.write(b"R")
            self.log.write(f"[DEBUG] Beginning R acquisition\n")
            
            ret = self.ser.read(5)
            self.log.write(f"[INFO] R acquisition successfully completed\n")
            return ret.hex()[2]+ret.hex()[3]

        def close_instrument(self):
            self.ser.close() 
            self.log.write(f"[INFO] Connection closed successfully\n \n")

In [13]:
if __name__ == '__main__' and ports:
    fpga = FPGA_communication(port = PORT, baud_rate = 115200)
    fpga.open_instrument()
    fpga.set_memory_addr()
    fpga.write_val_mem("55")
    fpga.display_mem_vals_leds()
    mem_val = fpga.read_mem()
    print(mem_val)
    fpga.close_instrument()

55
